# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

## Загрузка данных

In [1]:
#импортируем библиотеки

import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score



In [2]:
# прочитаем данные 

def data_reading(path):
    data = pd.read_csv(path)
    #print(f'Количество строк: {data.shape[0]}; \n\
#Количество столбцов: {data.shape[1]} \n')
    print(f'Информация о данных: \n')
    data.info()
    print(f'\nКоличество дубликатов: {data.duplicated().sum()} \n')
    print(f'\nДоля дубликатов: {data.duplicated().mean()}\n')
    print(f'\nКоличество пропусков:\n\
{data.isna().sum()} \n')
    print(f'\nДоля пропусков:\n\
{data.isna().mean()} \n')
    print(f'Первые 5 строк датасета: \n')
    display(data.head(5))
    return data

In [4]:
data = data_reading('insurance.csv')

Информация о данных: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB

Количество дубликатов: 153 


Доля дубликатов: 0.0306


Количество пропусков:
Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64 


Доля пропусков:
Пол                  0.0
Возраст              0.0
Зарплата             0.0
Члены семьи          0.0
Страховые выплаты    0.0
dtype: float64 

Первые 5 строк датасета: 



,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Почистим данные от имеющихся дубликтов

In [5]:
data = data.drop_duplicates().reset_index(drop = True)

In [6]:
# переименуем названия столбцов в соответствии с общепринятыми правилами

data = data.rename(columns = {'Пол':'gender',
                              'Возраст':'age', 
                              'Зарплата':'salary_payments',
                              'Члены семьи':'family_members',
                              'Страховые выплаты':'insurance_payments'  
                             })

Исходный датасет имеет размерность 5000 строк и 5 столбцов. 

В ходе обработки было выявлено 123 дубликата (или 3% от общего числа данных). Существенного влияния на результаты их наличие оказывать не должно, но для упрощения обучения модели они были исключены.

Пропуски в данных отсутсвуют, а значит можем приступать к решению поставленных задач.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Наша задача** - проверить, изменится ли качество линейной регрессии, определяемое среднеквадратичным отклонением (ошибкой) между предсказанием и целевым признаком. 

**Обоснование:** 

Новое предсказание: 

$$
a_1 = XPw_1
$$

В соответствии с теоретическим материалом (или примером выше):

$$
w_1 = ((XP)^T (XP))^{-1}(XP)^T y
$$

Для преобразования будем руководствоваться следущими свойствами матриц:

$$
AE = A
$$

$$
AA^{-1} = A^{-1}A = E $$ (для обратимых матриц)


где $E$ - единичная матрица;

$$
(AB)^{T} = B^T A^T
$$

$$
(A^{-1})^{T} = (A^{T})^{-1}
$$

$$
(AB)^{-1} = B^{-1} A^{-1}
$$

C учетом вышесказанного:

$$
w_1 = (P^TX^T X P)^{-1}P^T X^T y = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = 
$$

$$
a_1 = XPw_1 = XPP^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = X [PP^{-1}] (X^T X)^{-1} [(P P^{-1})^{T}] X^T y = (XE) (X^T X)^{-1} E^T X^T y = (X E) (X^T X)^{-1}  (XE)^T y = X (X^T X)^{-1}  X^T y = Xw = a
$$

По результатам преобразования можно заметить, что предсказания $a_1$ = $a$, соответственно качество линейной модели **не изменится**



Проверим наши результаты на практике

## Алгоритм преобразования

На основании преобразований в прошлом пункте теоретически подтвердилось, что при перемножении матрицы признаков на обратимую матрицу результат предсказания не изменится. 

На практике это позволит "зашифровать" исходные данные (признаки), не искажая эффективность предсказаний модели.

Для проверки реализуем следующий алгоритм: 

1) Создадим случайную квадратную матрицу $P$, размер которой равен количеству столбцов таблицы признаков (ширине матрицы признаков)

2) Проверим ее обратимость, перемножив ее с обратной матрицей $P^{-1}$. В случае, если в результате перемножения не получилась единичная матрица, вернемся в пункт 1.

3) Обучим модель линейной регрессии на новых данных и посчитаем вектов весов $w$, r2-метрику и сравним результаты с моделью линейной регрессии, обученной на исходных данных. Реузльтат r2 - метрики не должен исказиться.

## Проверка алгоритма

In [7]:
# разделим признаки на тренировочную и тестовую 

data_train, data_test = train_test_split(data, test_size = 0.25, random_state = 12345)

In [8]:
# определим признаки и целевой признак

features = data_train.drop('insurance_payments', axis=1)
target = data_train['insurance_payments']

test_features = data_test.drop('insurance_payments', axis=1)
test_target = data_test['insurance_payments']

In [9]:
model = LinearRegression()
model.fit(features,target)

print('Значение вектора веса линейной регрессии: {}'.format(model.coef_))
print('Значение свободного коэффициента: {}'.format(model.intercept_))
print('Значение r2-метрики: {}'.format(r2_score(test_target, model.predict(test_features))))

Значение вектора веса линейной регрессии: [ 1.45766002e-02  3.64782926e-02  1.79477716e-07 -1.23345013e-02]
Значение свободного коэффициента: -0.9767095974777156
Значение r2-метрики: 0.42307727492147296


In [10]:
# выполним срез с признаками из общего датасета для образования новых значений признаков 
# (путем перемножения исходной матрицы признаков на обратимую случайную матрицу)
new_features = data.drop('insurance_payments', axis=1) 

# создадим обратимую матрицу с порядком, равным ширине матрицы признаков

p = np.random.randint(100, size=(new_features.shape[1], new_features.shape[1]))
inv_p = np.linalg.inv(p)

print('Обратимая матрица P: \n', p, '\n')
print('Обратная матрица P^(-1): \n', inv_p, '\n')
print('Результат перемножения обратимой матрицы P на обратную P^(-1): \n', np.rint(np.dot(p, inv_p)), '\n')



Обратимая матрица P: 
 [[88 34 57 27]
 [56 46 12 65]
 [58 70 46  5]
 [31 93 32 39]] 

Обратная матрица P^(-1): 
 [[-0.01512829  0.02302298  0.03529522 -0.03242323]
 [-0.01686233  0.00632145  0.02026076 -0.00145936]
 [ 0.04288156 -0.03845012 -0.04972928  0.04077185]
 [ 0.01705034 -0.00182572 -0.03556578  0.02143953]] 

Результат перемножения обратимой матрицы P на обратную P^(-1): 
 [[ 1. -0.  0.  0.]
 [-0.  1.  0.  0.]
 [-0.  0.  1.  0.]
 [-0. -0.  0.  1.]] 



In [11]:
new_features = np.dot(new_features, p)
new_data = pd.DataFrame(new_features, columns = data.columns.drop('insurance_payments'))
new_data['insurance_payments'] = data['insurance_payments']

In [12]:
# определим признаки и целевой признак

data_train, data_test = train_test_split(new_data, test_size = 0.25, random_state = 12345)

features = data_train.drop('insurance_payments', axis=1)
target = data_train['insurance_payments']

test_features = data_test.drop('insurance_payments', axis=1)
test_target = data_test['insurance_payments']

In [13]:
model2 = LinearRegression()
model2.fit(features, target)

print('Значение вектора веса линейной регрессии: {}'.format(model2.coef_))
print('Значение свободного коэффициента: {}'.format(model2.intercept_))
print('Значение r2-метрики: {}'.format(r2_score(test_target, model2.predict(test_features))))

Значение вектора веса линейной регрессии: [ 1.01925067e-03  2.80433852e-06 -1.28043665e-03 -8.25154571e-05]
Значение свободного коэффициента: -0.9767095974756546
Значение r2-метрики: 0.4230772749214621


Как можно заметить, выводы полученные при преобразовании в теоретической части были **подтверждены**  практически.

При изменении значений в матрице признаков, изменилось значение вектора весов $w$. При этом величина r2-метрики **не изменилась**.